In [4]:
import csv
from itertools import repeat
import os
import pandas as pd
from pandas import read_csv, DataFrame, Index, MultiIndex
import copy
from IPython.display import display
import numpy as np
import collections
from itertools import dropwhile
import os
from subprocess import call

## - Set up Environment

In [5]:
TLC = ('/autofs/space/will_002/users/TLC/MGH/Subjects')
os.system('findsession -p bfr > FINDSESS.txt')
os.system('cd $TLC | touch subjlist.txt scan.txt unpacksdcmdir_1.csh unpacksdcmdir_2.csh')

transfer_scanlist=[]
subject_list=[]

unpack_log='unpack.log'
raw_cfg= 'cfg_raw.txt'
new_cfg='cfg.txt'

## - Generate Subject and Scan lists

In [6]:
##   NOTE: THIS WILL GENERATE SUBJECT AND         ##
##   SCAN LISTS FROM BOURGET - PROJECT CODE BFR   ##
##   MODIFY subjlist.txt AND scan.txt ACCORDINGLY ##

def get_scans_subjects(path):
    with open('%s/FINDSESS.txt' % path ,'r') as in_txt, \
        open('%s/subjlist.txt' % path ,'w') as sublist, \
        open('%s/scan.txt' % path ,'w') as scanlist:
        for line in in_txt:
                if 'SUBJECT' in line:
                    sub=line.strip('SUBJECT:  ')
                    sublist.write('%s \n' % sub.strip())
                    subject_list.append(sub)
                elif 'PATH' in line:
                    scan=line.strip('PATH   :')
                    scanlist.write('%s \n' % scan.strip())
                    transfer_scanlist.append(sub)
    return

get_scans_subjects(TLC)

## - unpackdcmsdir - 1

In [5]:
def unpacksdcmdir_step1(path):
    with open('%s/subjlist.txt' % path ,'r') as sublist, \
        open('%s/scan.txt' % path ,'r') as scanlist, \
        open('%s/unpacksdcmdir_1.csh' % path ,'w') as unpacksdcmdir_1:
        for scan,subject in zip(scanlist,sublist):
            unpacksdcmdir_1.write('unpacksdcmdir -src %s -targ %s/%s -scanonly %s/%s/%s \n' \
            % (scan.strip(), path, subject.strip(), path, subject.strip(), new_cfg))
        print "Next: source unpacksdcmdir_1.csh"
    return

unpacksdcmdir_step1(TLC)

#----------------------------#
# source .csh from Py - BETA #
#----------------------------#

# from subprocess import call
# os.system('source %s/unpacksdcmdir_1.csh' % TLC)

#  unpacksdcmdir_1 = 'unpacksdcmdir -src %s -targ %s/%s -scanonly %s/%s/%s' % (scan, TLC, subject, TLC, subject, new_cfg)
#  call(unpacksdcmdir_1,env=os.environ,shell=True)

Next: source unpacksdcmdir_1.csh


## - Create Configuration File

In [10]:
x2={'a':[],'b':[],'c':[],'d':[]}
def read_write_cfg(path,subject,unpack_log,raw_cfg): 
    infile=os.path.join(path,subject.strip(),unpack_log)
    outfile=os.path.join(path,subject.strip(),raw_cfg)
    with open(infile,'r') as in_txt, open(outfile,'w') as out_txt:
        for line in in_txt:
            if 'ok' in line:
                c=line.strip().split()
                n=str(c[0])
                scan=str(c[1])
                num=str(c[7])
                x2['a'].append(n)
                nifti='NIFTI'
                x2['b'].append(scan)
                x2['c'].append(nifti)
                x2['d'].append(num)

def format_cfg(path,subject,raw_cfg,new_cfg):
    with open('%s/subjlist.txt' % path ,'r') as sublist:
        for subject in sublist:
            subdir = os.path.join(path,"%s/cfg.txt" % subject.rstrip(' \n'))
            with open(subdir, 'w') as form_cfg:
                for a,b,c,d in zip(x2['a'],x2['b'],x2['c'],x2['d']):
                    form_cfg.write('\t'.join([a,b,c,d]) + '\n')
                print 'find cfg.txt formatted in subject %s directory' % subject
        return

## - Read Unpack Log and Write Configuration File

In [11]:
def iter_subjlist(path,subject_list,unpack_log,raw_cfg,new_cfg):
        with open('%s/subjlist.txt' % path ,'r') as sublist:
            for subject in sublist: 
                read_write_cfg(path,subject.strip(),unpack_log,raw_cfg)
                format_cfg(path,subject.strip(),raw_cfg,new_cfg)
            print(form_cfg)
        return

iter_subjlist(TLC, subject_list, unpack_log, raw_cfg, new_cfg)

find cfg.txt formatted in subject 02001MR01 
 directory
find cfg.txt formatted in subject test06 
 directory
find cfg.txt formatted in subject test07 
 directory
find cfg.txt formatted in subject test08 
 directory
find cfg.txt formatted in subject 020001MR01 
 directory
find cfg.txt formatted in subject 020003MR01 
 directory
find cfg.txt formatted in subject 020004MR01 
 directory
find cfg.txt formatted in subject 020008MR01 
 directory
find cfg.txt formatted in subject 020009MR01 
 directory
find cfg.txt formatted in subject 020005MR01 
 directory
find cfg.txt formatted in subject 020014MR01 
 directory
find cfg.txt formatted in subject 020006MR01 
 directory
find cfg.txt formatted in subject 020007MR01 
 directory
find cfg.txt formatted in subject 020011MR01 
 directory
find cfg.txt formatted in subject 020012MR01 
 directory
find cfg.txt formatted in subject 020013Mr01 
 directory
find cfg.txt formatted in subject 020015MR01 
 directory
find cfg.txt formatted in subject 020010MR01

find cfg.txt formatted in subject 020057MR01 
 directory
find cfg.txt formatted in subject 020058 
 directory
find cfg.txt formatted in subject 020055 
 directory
find cfg.txt formatted in subject MR01020056 
 directory
find cfg.txt formatted in subject MR01020058 
 directory
find cfg.txt formatted in subject MR01020058_2 
 directory
find cfg.txt formatted in subject 020059MR01 
 directory
find cfg.txt formatted in subject 020060MR01 
 directory
find cfg.txt formatted in subject 020026MR01 
 directory
find cfg.txt formatted in subject 020061MR01 
 directory
find cfg.txt formatted in subject 020065MR01 
 directory
find cfg.txt formatted in subject 020067MR01 
 directory
find cfg.txt formatted in subject 020069 
 directory
find cfg.txt formatted in subject 020070 
 directory
find cfg.txt formatted in subject 020071MR01 
 directory
find cfg.txt formatted in subject 020074MR01 
 directory
find cfg.txt formatted in subject 020075MR01 
 directory
find cfg.txt formatted in subject 020076MR01 

find cfg.txt formatted in subject 020032MR01 
 directory
find cfg.txt formatted in subject 020030 
 directory
find cfg.txt formatted in subject 020033MR01 
 directory
find cfg.txt formatted in subject 020034 
 directory
find cfg.txt formatted in subject 020035MR01 
 directory
find cfg.txt formatted in subject 0020036MR01 
 directory
find cfg.txt formatted in subject 020038MR01 
 directory
find cfg.txt formatted in subject 020039MR01 
 directory
find cfg.txt formatted in subject 020040MR01 
 directory
find cfg.txt formatted in subject 020041MR01 
 directory
find cfg.txt formatted in subject 020042 
 directory
find cfg.txt formatted in subject 020043MR01 
 directory
find cfg.txt formatted in subject 020044MR01 
 directory
find cfg.txt formatted in subject 020046MR01 
 directory
find cfg.txt formatted in subject 020045MR01 
 directory
find cfg.txt formatted in subject 020048MR01 
 directory
find cfg.txt formatted in subject 020047MR01 
 directory
find cfg.txt formatted in subject 020049MR

find cfg.txt formatted in subject 02079MR01 
 directory
find cfg.txt formatted in subject 02086MR01 
 directory
find cfg.txt formatted in subject 02089MR01 
 directory
find cfg.txt formatted in subject 02085MR01 
 directory
find cfg.txt formatted in subject 02090MR01 
 directory
find cfg.txt formatted in subject 020088MR01 
 directory
find cfg.txt formatted in subject 020092MR01 
 directory
find cfg.txt formatted in subject 020094MR01 
 directory
find cfg.txt formatted in subject 02093MR01 
 directory
find cfg.txt formatted in subject 02096MR01 
 directory
find cfg.txt formatted in subject 020097MR01 
 directory
find cfg.txt formatted in subject 020094MR01 
 directory
find cfg.txt formatted in subject 020095MR01 
 directory
find cfg.txt formatted in subject 020098MR01 
 directory
find cfg.txt formatted in subject ep016 
 directory
find cfg.txt formatted in subject 02001MR01 
 directory
find cfg.txt formatted in subject test06 
 directory
find cfg.txt formatted in subject test07 
 direc

find cfg.txt formatted in subject 020039MR01 
 directory
find cfg.txt formatted in subject 020040MR01 
 directory
find cfg.txt formatted in subject 020041MR01 
 directory
find cfg.txt formatted in subject 020042 
 directory
find cfg.txt formatted in subject 020043MR01 
 directory
find cfg.txt formatted in subject 020044MR01 
 directory
find cfg.txt formatted in subject 020046MR01 
 directory
find cfg.txt formatted in subject 020045MR01 
 directory
find cfg.txt formatted in subject 020048MR01 
 directory
find cfg.txt formatted in subject 020047MR01 
 directory
find cfg.txt formatted in subject 020049MR01 
 directory
find cfg.txt formatted in subject 020062MR01 
 directory
find cfg.txt formatted in subject 020051MR01 
 directory
find cfg.txt formatted in subject 020052MR01 
 directory
find cfg.txt formatted in subject 020053MR01 
 directory
find cfg.txt formatted in subject 020054MR01 
 directory
find cfg.txt formatted in subject 020057MR01 
 directory
find cfg.txt formatted in subject 0

...

find cfg.txt formatted in subject 02096MR01 
 directory
find cfg.txt formatted in subject 020097MR01 
 directory
find cfg.txt formatted in subject 020094MR01 
 directory
find cfg.txt formatted in subject 020095MR01 
 directory
find cfg.txt formatted in subject 020098MR01 
 directory
find cfg.txt formatted in subject ep016 
 directory
find cfg.txt formatted in subject 02001MR01 
 directory
find cfg.txt formatted in subject test06 
 directory
find cfg.txt formatted in subject test07 
 directory
find cfg.txt formatted in subject test08 
 directory
find cfg.txt formatted in subject 020001MR01 
 directory
find cfg.txt formatted in subject 020003MR01 
 directory
find cfg.txt formatted in subject 020004MR01 
 directory
find cfg.txt formatted in subject 020008MR01 
 directory
find cfg.txt formatted in subject 020009MR01 
 directory
find cfg.txt formatted in subject 020005MR01 
 directory
find cfg.txt formatted in subject 020014MR01 
 directory
find cfg.txt formatted in subject 020006MR01 
 dir

find cfg.txt formatted in subject 020043MR01 
 directory
find cfg.txt formatted in subject 020044MR01 
 directory
find cfg.txt formatted in subject 020046MR01 
 directory
find cfg.txt formatted in subject 020045MR01 
 directory
find cfg.txt formatted in subject 020048MR01 
 directory
find cfg.txt formatted in subject 020047MR01 
 directory
find cfg.txt formatted in subject 020049MR01 
 directory
find cfg.txt formatted in subject 020062MR01 
 directory
find cfg.txt formatted in subject 020051MR01 
 directory
find cfg.txt formatted in subject 020052MR01 
 directory
find cfg.txt formatted in subject 020053MR01 
 directory
find cfg.txt formatted in subject 020054MR01 
 directory
find cfg.txt formatted in subject 020057MR01 
 directory
find cfg.txt formatted in subject 020058 
 directory
find cfg.txt formatted in subject 020055 
 directory
find cfg.txt formatted in subject MR01020056 
 directory
find cfg.txt formatted in subject MR01020058 
 directory
find cfg.txt formatted in subject MR010

find cfg.txt formatted in subject 020094MR01 
 directory
find cfg.txt formatted in subject 020095MR01 
 directory
find cfg.txt formatted in subject 020098MR01 
 directory
find cfg.txt formatted in subject ep016 
 directory
find cfg.txt formatted in subject 02001MR01 
 directory
find cfg.txt formatted in subject test06 
 directory
find cfg.txt formatted in subject test07 
 directory
find cfg.txt formatted in subject test08 
 directory
find cfg.txt formatted in subject 020001MR01 
 directory
find cfg.txt formatted in subject 020003MR01 
 directory
find cfg.txt formatted in subject 020004MR01 
 directory
find cfg.txt formatted in subject 020008MR01 
 directory
find cfg.txt formatted in subject 020009MR01 
 directory
find cfg.txt formatted in subject 020005MR01 
 directory
find cfg.txt formatted in subject 020014MR01 
 directory
find cfg.txt formatted in subject 020006MR01 
 directory
find cfg.txt formatted in subject 020007MR01 
 directory
find cfg.txt formatted in subject 020011MR01 
 di

find cfg.txt formatted in subject 020049MR01 
 directory
find cfg.txt formatted in subject 020062MR01 
 directory
find cfg.txt formatted in subject 020051MR01 
 directory
find cfg.txt formatted in subject 020052MR01 
 directory
find cfg.txt formatted in subject 020053MR01 
 directory
find cfg.txt formatted in subject 020054MR01 
 directory
find cfg.txt formatted in subject 020057MR01 
 directory
find cfg.txt formatted in subject 020058 
 directory
find cfg.txt formatted in subject 020055 
 directory
find cfg.txt formatted in subject MR01020056 
 directory
find cfg.txt formatted in subject MR01020058 
 directory
find cfg.txt formatted in subject MR01020058_2 
 directory
find cfg.txt formatted in subject 020059MR01 
 directory
find cfg.txt formatted in subject 020060MR01 
 directory
find cfg.txt formatted in subject 020026MR01 
 directory
find cfg.txt formatted in subject 020061MR01 
 directory
find cfg.txt formatted in subject 020065MR01 
 directory
find cfg.txt formatted in subject 020

NameError: global name 'form_cfg' is not defined

## - unpackdcmsdir - 2

In [15]:
with open('%s/unpacksdcmdir_2.csh' % TLC ,'w') as unpacksdcmdir_2, \
open('%s/subjlist.txt' % TLC ,'r') as sublist:
    subdir = os.path.join(TLC,"%s/cfg.txt" % subject.rstrip(' \n'))
    for scan,subject in zip(transfer_scanlist,sublist):        
        unpacksdcmdir_2.write('unpacksdcmdir -src %s -targ %s/%s -fsfast -cfg %s \n'\
        % (scan.rstrip(' \n'), TLC.rstrip(' \n'), subject.rstrip(' \n'), subdir))
    print "Next: source unpacksdcmdir_2.csh"

NameError: name 'subject' is not defined

## APPENDIX / MISC

### DICOM to NIFTI

In [ ]:
scan_list=['fieldmap_BOLD','AAScout', 'DIFFUSION_HighRes_REVISED', 'fieldmap_BOLD', \
           'fieldmap_generic_DTI', 'fMRI_ASSOCIATIVE_BOLD', 'fMRI_resting_state_1', \
           'fMRI_resting_state_2', 'fMRI_REWARD_BOLD', 'localizer', 'mprage_1mm_iso']

output=[]

def dicom2nii_get_path(path):
    for subject in subject_list:
        if subject in os.listdir(path):
            for scan in scan_list:
                if scan in os.listdir(os.path.join(path,subject)):
                    insert=os.path.join(path, subject, scan)
                    subdir=os.listdir(insert)
                    for item in subdir:
                        if item.startswith('0'):
                            insert2=os.path.join(path, subject, scan, item)
                            dicom1=os.listdir(insert2)[0]
                            insert3=os.path.join(path, subject, scan, item, dicom1)
                            save_me=output.append(insert3)  
    return

dicom2nii_get_path('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH/')

### Read Configuration File

In [ ]:
def read_cfg(path,subjid,doc): 
    insert=os.path.join(path, subjid, doc)
    doc=open(insert,'r' )
    print doc.read()
    
read_cfg('/autofs/space/will_001/users/TLC/pilot_1st20subj/MGH/','020043MR01','unpack.log')

### Read unpack.log and write cfg.txt file 
##### Note: Iterates through subjects txt file in lieu of array

In [12]:
def iter_subjlist(path,subject_list,unpack_log,raw_cfg,new_cfg):
    subjects=os.path.join(path,subject_list)
    with open(subjects,'U') as slist:
        for subject in slist: 
            subject=subject.strip().split()
            print(subject)
            read_write_cfg(path,subject,unpack_log,raw_cfg)
            format_cfg(path,subject,raw_cfg,new_cfg)
    return